# Graph Classification

AutoGL supports multiple graph related tasks, including graph classification.

In this file we will give you a simple example to show how to use AutoGL to do the graph classification task.

## Import libraries

First, you should import some libraries and you can set the random seed before you split the dataset and train the model.

In [ ]:
import random
import torch
import numpy as np
import sys, os
sys.path.insert(0, '../../')
from autogl.datasets import build_dataset_from_name, utils
from autogl.solver import AutoGraphClassifier
from autogl.module import Acc
from argparse import ArgumentParser, ArgumentDefaultsHelpFormatter
from autogl.backend import DependentBackend
if DependentBackend.is_pyg():
    from autogl.datasets.utils.conversion import to_pyg_dataset as convert_dataset
else:
    from autogl.datasets.utils.conversion import to_dgl_dataset as convert_dataset

backend = DependentBackend.get_backend_name()

## Config

In [ ]:
from argparse import Namespace
args_dict = {'dataset': 'mutag', 'configs': '../../configs/graphclf_gin_benchmark.yml', 'device': 5, 'seed': 0}
args = Namespace(**args_dict)

if args.device == -1:
    args.device = "cpu"

if torch.cuda.is_available() and args.device != "cpu":
    torch.cuda.set_device(args.device)
seed = args.seed
# set random seed
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

## Load Dataset

AutoGL provides a very convenient interface to obtain and partition common datasets, such as mutag, imdb, and ptc_mr, etc.

You just need to give the name of the dataset you want and AutoGL will return the dataset.

In this example, we evaluate model on mutag dataset in the graph classification task.

In [ ]:
dataset = build_dataset_from_name(args.dataset)
_converted_dataset = convert_dataset(dataset)
if args.dataset.startswith("imdb"):
    from autogl.module.feature import OneHotDegreeGenerator

    if DependentBackend.is_pyg():
        from torch_geometric.utils import degree
        max_degree = 0
        for data in _converted_dataset:
            deg_max = int(degree(data.edge_index[0], data.num_nodes).max().item())
            max_degree = max(max_degree, deg_max)
    else:
        max_degree = 0
        for data, _ in _converted_dataset:
            deg_max = data.in_degrees().max().item()
            max_degree = max(max_degree, deg_max)
    dataset = OneHotDegreeGenerator(max_degree).fit_transform(dataset, inplace=False)
elif args.dataset == "collab":
    from autogl.module.feature._auto_feature import OnlyConstFeature

    dataset = OnlyConstFeature().fit_transform(dataset, inplace=False)
utils.graph_random_splits(dataset, train_ratio=0.8, val_ratio=0.1, seed=args.seed)

## Initialize Solver

After obtaining the dataset, we need to initialize the model.

However, as AutoGL provides a convenient method to use HPO to better optimize the model, we can train the model through the solver class provided by AutoGL.

Solver in AutoGL usually uses a config file for lazy initialization. The format of the config file can be found in the ../config folder for examples, or you can read our tutorial for some help.

In [ ]:
autoClassifier = AutoGraphClassifier.from_config(args.configs)

## Train

After the initialization is finished, you can use the interface provided by AutoGL to optimize the model through HPO.

In [ ]:
# train
autoClassifier.fit(dataset, evaluation_method=[Acc], seed=args.seed)
autoClassifier.get_leaderboard().show()

print("best single model:\n", autoClassifier.get_leaderboard().get_best_model(0))

## Evaluation

After training, you can evaluate the model.

In [ ]:
# test
acc = autoClassifier.evaluate(metric="acc")
print("test acc {:.4f}".format(acc))